In [107]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
import numpy as np

dataset = pd.read_csv('dataset_LUMINAL_A_B.csv')

X, y = dataset.loc[:, dataset.columns != 'l'], dataset.loc[:, dataset.columns == 'l']

scaler = StandardScaler()
X = scaler.fit_transform(X)


encoder = OrdinalEncoder()
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)




In [32]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [125]:
model = Sequential([
    Dense(1022, activation='relu'),
    Dropout(0.3),
    Dense(150, activation='relu'),
    Dropout(0.2),
    Dense(150, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train, y_train,epochs=50)


Epoch 1/50
3/3 [==============================] - 0s 4ms/step - loss: 0.7332 - accuracy: 0.5500
Epoch 2/50
3/3 [==============================] - 0s 4ms/step - loss: 0.7004 - accuracy: 0.5125
Epoch 3/50
3/3 [==============================] - 0s 5ms/step - loss: 0.6842 - accuracy: 0.5750
Epoch 4/50
3/3 [==============================] - 0s 20ms/step - loss: 0.6365 - accuracy: 0.6250
Epoch 5/50
3/3 [==============================] - 0s 6ms/step - loss: 0.6757 - accuracy: 0.5750
Epoch 6/50
3/3 [==============================] - 0s 6ms/step - loss: 0.5936 - accuracy: 0.6625
Epoch 7/50
3/3 [==============================] - 0s 9ms/step - loss: 0.5765 - accuracy: 0.7625
Epoch 8/50
3/3 [==============================] - 0s 5ms/step - loss: 0.5989 - accuracy: 0.6500
Epoch 9/50
3/3 [==============================] - 0s 4ms/step - loss: 0.5063 - accuracy: 0.8125
Epoch 10/50
3/3 [==============================] - 0s 3ms/step - loss: 0.5009 - accuracy: 0.8250
Epoch 11/50
3/3 [=====================

In [127]:
import numpy as np

y_pred = model.predict(X_test)




from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, np.rint(y_pred)))


1/1 [==============================] - 0s 14ms/step
0.8
